In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline 
from functions import hyperSearch,createGraphRNN2,RNN_generator,init_state_update,RNN_forecast_Repeat
import cPickle
from hyperopt import hp, fmin, tpe

In [2]:
discreteList = ['dayOfWeek','payDay','month','earthquake','type','locale','locale_name','transferred','onpromotion']

In [3]:
learningRate = 1e-4
epoch = 30
bucketSize = 10000

In [4]:
cardinalitys_X = [55, 4001, 34, 337, 2, 23, 17, 6, 18]
cardinalitys_T = [7, 2, 13, 2, 7, 4, 25, 2, 3]
dimentions_X = [2, 20, 1, 2, 1, 1, 1, 1, 1]
dimentions_T = [1, 1, 1, 1, 1, 1, 1, 1, 1]

In [5]:
dX = sum(dimentions_X)
dT = sum(dimentions_T)
d = dX + dT + 2 # 2 for two cont variables

In [6]:
prefix = 'train'
y_np = np.loadtxt(prefix+'_Y',dtype=np.float32, delimiter=",") 
weight_np = np.loadtxt(prefix+'_Weight',dtype=np.float32, delimiter=",") 
Con_np = np.loadtxt(prefix+'_Con',dtype=np.float32, delimiter=",") 
X_np = np.loadtxt(prefix+'_X',dtype=np.int32,delimiter=",") 
Count_np = np.loadtxt(prefix+'_Count',dtype=np.int32,delimiter=",") 
Dis_np = [np.loadtxt(prefix+'_Dis'+str(j),dtype=np.int32,delimiter=",")  for j in range(len(discreteList))]

In [7]:
prefix = 'val_SI'
y_np_val = np.loadtxt(prefix+'_Y',dtype=np.float32, delimiter=",") 
weight_np_val = np.loadtxt(prefix+'_Weight',dtype=np.float32, delimiter=",") 
Con_np_val = np.loadtxt(prefix+'_Con',dtype=np.float32, delimiter=",") 
X_np_val = np.loadtxt(prefix+'_X',dtype=np.int32,delimiter=",") 
Count_np_val = np.loadtxt(prefix+'_Count',dtype=np.int32,delimiter=",") 
Dis_np_val = [np.loadtxt(prefix+'_Dis'+str(j),dtype=np.int32,delimiter=",")  for j in range(len(discreteList))]

In [8]:
index = np.loadtxt('Index_val',dtype=np.int32,delimiter=",") 

In [9]:
space = hp.choice('batch_seq',[[100,16],[50,32],[25,64]]) + \
        [hp.uniform('keep_prob',0.4,0.9),hp.quniform('n_layers',2,5,1),\
         hp.uniform('grad_clip',1,10),\
         hp.choice('cell_type',['residual','highway','NormLSTM']),\
         hp.uniform('downsample',0.1,0.95),\
         hp.choice('optimizer',['SGD','Adam']),\
         hp.choice('actFun',['tanh','relu'])]

In [80]:
    sess = tf.InteractiveSession()
    sess.run(tf.global_variables_initializer())

In [83]:
X_nps =RNN_generator(y_np, weight_np,Con_np,Dis_np,X_np,Count_np,\
                                  batch_size,seq_len,bucketSize,downSample=downsample).next()

In [88]:
init_state = sess.run(state,\
                                 dict(zip(inputs,X_nps+[learningRate,init_state])))

In [86]:
init_state = tuple([tf.contrib.rnn.LSTMStateTuple(np.zeros((batch_size,d),dtype=np.float32),\
                                                      np.zeros((batch_size,d),dtype=np.float32))\
                                                        for i in range(n_layers)])

In [92]:
init_state[0].c

array([[ 1.10766649,  0.63062775, -0.21056104, ...,  0.46736917,
        -0.71063995,  0.88273227],
       [ 1.63793659,  0.92491907, -0.31394011, ..., -0.34325564,
        -0.23101175, -0.58160162],
       [-0.52010763, -0.8445369 , -0.75679171, ...,  0.03149073,
         1.37349415, -0.71118367],
       ..., 
       [-0.59264266, -0.88014412, -0.02659364, ..., -0.36313671,
         0.46662873, -0.32815546],
       [-0.29076535, -0.45315903, -0.04426862, ..., -0.45056081,
         0.13092676, -0.21516006],
       [ 2.81334329, -0.19215184,  1.76722324, ..., -0.23067547,
        -0.36292753,  0.44495115]], dtype=float32)

In [20]:
best = fmin(hyperSearch,space,tpe.suggest,100)

TypeError: 'Tensor' object is not iterable.

In [19]:
def hyperSearch(paras):   
    batch_size,seq_len,keep_prob,n_layers,grad_clip,cell_type,downsample,optimizer,actFun = \
	paras[0],paras[1],paras[2],int(paras[3]),paras[4],paras[5],paras[6],paras[7],paras[8]
    # training 
    inputs,train_op,cost,saver,yhat,state = createGraphRNN2(batch_size,seq_len,cardinalitys_X,cardinalitys_T,\
                                                    dimentions_X,dimentions_T,dX,d,keep_prob,n_layers,\
                                                    grad_clip,cell_type,optimizer,actFun)
    sess = tf.InteractiveSession()
    sess.run(tf.global_variables_initializer())
    init_state = tuple([np.zeros((batch_size,d),dtype=np.float32) for i in range(n_layers)])
    for i in range(epoch*100/batch_size):
        for j,X_nps in enumerate(RNN_generator(y_np, weight_np,Con_np,Dis_np,X_np,Count_np,\
                                  batch_size,seq_len,bucketSize,downSample=downsample)):
            _,init_state = sess.run([train_op,state],\
                                 dict(zip(inputs,X_nps+[learningRate,init_state])))
    saver.save(sess,'RNN_fillin_01')
        
    # testing        
    inputs,train_op,cost,saver,yhat,state = createGraphRNN2(None,1,cardinalitys_X,cardinalitys_T,\
                                            dimentions_X,dimentions_T,dX,d,keep_prob,n_layers,\
                                            grad_clip,cell_type,optimizer,actFun)
    sess = tf.InteractiveSession()
    sess.run(tf.global_variables_initializer())
    saver.restore(sess,'RNN_fillin_01')
    init_tot_list = init_state_update(sess,inputs,state,batch_size*10,d,n_layers,\
                          y_np[index],Con_np[index],X_np[index],Count_np[index],\
                          [dis[index] for dis in Dis_np])
    y_val_hat = RNN_forecast_Repeat(10,sess,inputs,state,yhat,batch_size*10,n_layers,\
                                    np.expand_dims(y_np[index,Count_np[index]-1],-1),\
                                    Con_np_val,X_np_val,Dis_np_val,init_tot_list)
    loss = loss_func(weight_np_val[:,np.newaxis],y_val_hat,y_np_val)    
    print "loss:{} ,batch_size:{} ,seq_len:{} ,keep_prob:{} ,n_layers:{} ,grad_clip:{} ,cell_type:{} ,downsample:{} ,optimizer:{} ,actFun:{} \n"\
          .format(loss,batch_size,seq_len,keep_prob,n_layers,grad_clip,cell_type,downsample,optimizer,actFun)
    return 100 if (np.isnan(loss) or np.isinf(loss)) else loss